# Preprocessing 

In [1]:
import pandas as pd
import glob
import os
from pathlib import Path
import re
import pickle
import string
import numpy as np
from striprtf.striprtf import rtf_to_text

## Loading files into the Data Frame

In [18]:
# Define the directory path where RTF files are located
dir_path = "../data/sec_POL/"

def get_docx_files(dir_path):
    """
    Retrieves a list of RTF file paths within a specified directory and its subdirectories.

    Parameters:
    dir_path (str): The directory path to search for RTF files.

    Returns:
    list: A list of file paths to RTF files found within the directory and its subdirectories.
    """
    docx_files = []
    
    for root, _, files in os.walk(dir_path):
        for f in files:
            if f.endswith('.rtf'):
                docx_files.append(os.path.join(root, f))
    
    return docx_files

# Get a list of RTF files in the specified directory and its subdirectories
docx_files = get_docx_files(dir_path)

# Create an empty list to store data extracted from the RTF files
data = []

# Iterate over all RTF files in the directory
for filename in docx_files:
    if filename.endswith(".rtf"):
        with open(filename, 'rb') as file:
            rtf_content = file.read().decode('utf-8')  # Decode binary content to string
            
            # Assuming you have a function rtf_to_text that converts RTF to plain text
            rtf_text = rtf_to_text(rtf_content)
            
            # Append the filename (excluding the ".rtf" extension) and the extracted text to the data list
            data.append([filename[:-5], rtf_text])

# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(data, columns=['f_name', 'raw_txt'])

In [19]:
len(df)

12

In [20]:
df['raw_txt_l']=df['raw_txt'].apply(lambda x: x.split('\n\n\n\n\n\n'))

In [21]:
df=df.explode('raw_txt_l')

In [22]:
df

,f_name,raw_txt,raw_txt_l
0,../data/sec_POL/Fakt/Factiva-20230808-125,\nKsiądz Janusz Koplewski (59 l.): Odwagi! Wi...,\nKsiądz Janusz Koplewski (59 l.): Odwagi! Wi...
1,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,\n\n\nKraj bez Boga\n\n523 Wörter\n8 Januar 2...,\n\n\nKraj bez Boga\n\n523 Wörter\n8 Januar 2...
1,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,\n\n\nKraj bez Boga\n\n523 Wörter\n8 Januar 2...,Jaka Polska po rządach PiS?\n\n861 Wörter\n12 ...
1,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,\n\n\nKraj bez Boga\n\n523 Wörter\n8 Januar 2...,"Watykan, Rosja, Chiny. W co naprawdę gra Franc..."
1,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,\n\n\nKraj bez Boga\n\n523 Wörter\n8 Januar 2...,"Górski & Cieślik: Dokąd jechać, dokąd nie i dl..."
...,...,...,...
11,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,\n\n\nZałożyciel Kościoła Shincheonji przepra...,"Przedstawiciel ""chamskiej hołoty"": Kaczyński n..."
11,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,\n\n\nZałożyciel Kościoła Shincheonji przepra...,Cztery nowe ogniska koronawirusa na Podkarpaci...
11,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,\n\n\nZałożyciel Kościoła Shincheonji przepra...,Koronawirus. Dwaj księża z Grabówki w kwaranta...
11,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,\n\n\nZałożyciel Kościoła Shincheonji przepra...,Sanepid robi testy na koronawirusa po procesji...


## text cleaning

In [23]:
def clean(text):
    text = re.sub("http\S+|www\S+|fb.com\S+|Sunday Life\S+", '', text)
    text = ''.join([char for char in text if char not in '–!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”·‘'])
    text = re.sub('\n',' ',text)
    text = re.sub('‹|›|„','',text)
    text = re.sub(r'\b\d+\b\s*$', '', text)
    text = re.sub(r'\b\w*\d+\w*\b', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r'•\t', '', text)
    return text.strip()

In [24]:
df['cleaned_txt'] = df['raw_txt_l'].apply(clean)

In [25]:
df['len_text'] = df['cleaned_txt'].apply(lambda x: len(x.split()))

In [26]:
# sprawdzam jak długi jest tekst i czy czegoś nie ucięło, wyrywkowo
df['len_text'].describe()

count      876.000000
mean      1272.804795
std       1235.594816
min          0.000000
25%        538.000000
50%        898.500000
75%       1672.500000
max      12144.000000
Name: len_text, dtype: float64

In [27]:
df=df[df['len_text']>300]

In [28]:
di={'Polnisch':'','Copyright 2020. Ringier Axel Springer Polska':'','Fakt':'','FKTPO':'','©020, Agora. All rights reserved.':'','GAZWYB':'','Gazeta Wyborcza & Wyborcza.pl':'',
   'Rzeczpospolita':'','www.rp.pl':'','RZEPOL':''}
df['cleaned_txt'] = df['cleaned_txt'].replace(di, regex=True)

/var/folders/7q/c3c06q9s7m552ts_r68zhmz00000gn/T/ipykernel_7742/2084731100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_txt'] = df['cleaned_txt'].replace(di, regex=True)


In [29]:
df=df.reset_index(drop=True)

In [30]:
df['cleaned_txt'][0]

'Ksiądz Janusz Koplewski l Odwagi Wirus to nie kara boska ROZMAWIAŁ DONAT SZYLLER Wörter März    Copyright Ringier Axel Springer Polska FAKT Część wiernych mówi o koronawirusie jako o karze boskiej Jak ksiądz uważa słusznie Ks Janusz Koplewski Oczywiście że nie Bóg jest miłością więc jak może nas karać On czuwa nad wszystkim i chce nas zbawić On umarł z miłości do nas więc jak może nas karać Jezus mówi nam przecież Odwagi Ja jestem nie bójcie się Jak w tych trudnych dniach powinien postępować człowiek wierzący Powinien zachować roztropność i higienę Stosować się do zaleceń służb państwowych myć dokładnie ręce unikać tłumów jeśli to możliwe ograniczyć wychodzenie z domu Jako chrześcijanin pamiętam o tym że mam i duszę i ciało więc o to ciało też muszę zadbać Jest przykazanie nie zabijaj więc nie mogę nieroztropnością doprowadzić do zabicia siebie Bardzo ważna jest też oczywiście sfera duchowa Powinien pokładać ufność w Bogu i nie ulegać strachowi Strach jest dziś w nas wszystkich jeśli 

In [36]:
possible_words = ['pandemia', 'COVID', 'covid','COVID-19','covid-19','wirus','Wirus','Pandemia','pandemii']

df = df[df['cleaned_txt'].str.contains('|'.join(possible_words))]

In [37]:
df=df[['f_name','cleaned_txt']]

In [39]:
df.to_csv('../data/clean/sec_pol_clean.csv')

In [38]:
df

,f_name,cleaned_txt
0,../data/sec_POL/Fakt/Factiva-20230808-125,Ksiądz Janusz Koplewski l Odwagi Wirus to nie ...
1,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,Kraj bez Boga Wörter Januar © Copyright Al...
2,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,Jaka Polska po rządach PiS Wörter Januar © ...
3,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,Watykan Rosja Chiny W co naprawdę gra Francisz...
4,../data/sec_POL/Rzeczpospolita/Factiva-2023080...,Górski Cieślik Dokąd jechać dokąd nie i dlacze...
...,...,...
813,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,Katolicy w tłumie modlą się o wygaszenie pande...
814,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,Robert Biedroń Jestem jedynym kandydatem zmian...
816,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,Cztery nowe ogniska koronawirusa na Podkarpaci...
817,../data/sec_POL/Gazeta Wybrocza/Factiva-202308...,Koronawirus Dwaj księża z Grabówki w kwarantan...
